# Connect Colab to a Chameleon server

This notebook describes how to connect Colab to a server running on Chameleon. This allows you to run experiments requiring bare metal access, storage, memory, GPU and compute that exceeds the abilities of Colab's hosted runtime, but with Colab's familiar interface (and notebooks stored in your Google Drive). It also allows you to easily go back and forth between the convenience of Colab's hosted runtime and Chameleon's greater capabilities, depending on the needs of your experiment.

## Provision the resource


### Check resource availability

This notebook will try to reserve a bare metal Ubuntu server with RTX6000 GPU on CHI@UC - pending availability. Before you begin, you should check the host calendar at [https://chi.uc.chameleoncloud.org/project/leases/calendar/host/](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/). In the "Node Type" dropdown, filter on `gpu_rtx_6000` and make sure some hosts are available.

### Chameleon configuration

You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

In [10]:
import chi, os, time
from chi import lease
from chi import server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME') # change this if you need to
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)
username = os.getenv('USER') # all exp resources will have this prefix

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


If you need to change the details of the Chameleon server, e.g. use a different GPU type depending on availability, you can do that in the following cell.

In [11]:
# NODE_TYPE = "gpu_rtx_6000"
NODE_TYPE = "compute_skylake"

### Reservation

The following cell will create a reservation that begins now, and ends in 8 hours. You can modify the start and end date as needed. 

If there is no compatible device available right now, you can refer to the [CHI@UC host calendar](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/) (change the "Node type" selection to `gpu_rtx_6000`). Set the start and end time according to availability. Then,

* run the cell below to reserve your GPU node for some time in the future
* at the beginning of your (future) reservation, you'll run this notebook again but you will *skip* the "Reservation" section

In [12]:
res = []
lease.add_node_reservation(res, node_type=NODE_TYPE, count=1)

start_date, end_date = lease.lease_duration(days=0, hours=8)
# if you won't start right now - comment the line above, uncomment two lines below
# start_date = "2023-01-01 00:00" # manually define to desired start time 
# end_date =   "2023-01-01 08:00" # manually define to desired start time 

l = lease.create_lease(f"colab-{username}-{NODE_TYPE}", res, start_date=start_date, end_date=end_date)
l = lease.wait_for_active(l["id"])

### Provisioning resources

This section provisions resources. It will take approximately 10 minutes. You can check on its status in the Chameleon web-based UI: [https://chi.uc.chameleoncloud.org/project/instances/](https://chi.uc.chameleoncloud.org/project/instances/), then come back here when it is in the READY state.

In [13]:
# continue here, whether using a lease created just now or one created earlier
l = lease.get_lease(f"colab-{username}-{NODE_TYPE}")

In [14]:
reservation_id = lease.get_node_reservation(l["id"])
server.create_server(
    f"colab-{username}-{NODE_TYPE}", 
    reservation_id=reservation_id,
    image_name="CC-Ubuntu22.04"
)
server_id = server.get_server_id(f"colab-{username}-{NODE_TYPE}")
server.wait_for_active(server_id)

openstack.compute.v2.server.Server(id=49c60e1f-0b13-45a6-93ed-6823af79ad56, name=colab-tcz212_nyu_edu-compute_skylake, status=ACTIVE, tenant_id=cb970a4b0f2e42c9b1b3f9015d02f8a5, user_id=1f918ad6fda83a67301f1f51fdeca66420f01ebaa7bb42d69857bd2821bee5b2, metadata={}, hostId=c9f7f1389ac96a530d6c179791ae187dfcdcf6acd066d7d067c56aa6, image={'id': 'c6122262-2829-47ae-b8bc-046a1300d46c', 'links': [{'rel': 'bookmark', 'href': 'https://chi.uc.chameleoncloud.org:8774/images/c6122262-2829-47ae-b8bc-046a1300d46c'}]}, flavor={'vcpus': 1, 'ram': 1, 'disk': 20, 'ephemeral': 0, 'swap': 0, 'original_name': 'baremetal', 'extra_specs': {'resources:CUSTOM_BAREMETAL': '1', 'resources:VCPU': '0', 'resources:MEMORY_MB': '0', 'resources:DISK_GB': '0'}}, created=2024-10-29T22:31:48Z, updated=2024-10-29T22:38:50Z, addresses={'sharednet1': [{'version': 4, 'addr': '10.140.83.25', 'OS-EXT-IPS:type': 'fixed', 'OS-EXT-IPS-MAC:mac_addr': '24:6e:96:79:1a:22'}]}, accessIPv4=, accessIPv6=, links=[{'rel': 'self', 'href': 

Associate an IP address with this server:

In [15]:
reserved_fip = server.associate_floating_ip(server_id)

and wait for it to come up:

In [16]:
server.wait_for_tcp(reserved_fip, port=22)

## Install stuff

The following cells will install some basic packages in order to connect your Colab frontend to your Chameleon server. However, you may want to log in to your Chameleon server in order to access its terminal and install or configure packages outside of Colab.

To log in to the resource, use File > New > Terminal in the Chameleon JupyterHub environment, or your local terminal, and run:


In [39]:
print("ssh cc@" + reserved_fip)

ssh cc@192.5.87.106


Meanwhile, install an updated CUDA, Python and JupyterHub on your resource:

In [40]:
from chi import ssh

node = ssh.Remote(reserved_fip)

In [41]:
node.run('sudo apt update')
node.run('sudo apt -y install python3-pip python3-dev')
node.run('sudo pip3 install --upgrade pip')

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 270 kB in 1s (227 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
1 package can be upgraded. Run 'apt list --upgradable' to see it.


Reading package lists...
Building dependency tree...
Reading state information...
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
7 not fully installed or removed.
After this operation, 0 B of additional disk space will be used.
Setting up nvidia-dkms-545 (545.23.08-0ubuntu1) ...
update-initramfs: deferring update (trigger activated)

A modprobe blacklist file has been created at /etc/modprobe.d to prevent Nouveau
from loading. This can be reverted by deleting the following file:
/etc/modprobe.d/nvidia-graphics-drivers.conf

A new initrd image has also been created. To revert, please regenerate your
initrd by running the following command after deleting the modprobe.d file:
`/usr/sbin/initramfs -u`

*****************************************************************************
*** Reboot your computer and verify that the NVIDIA graphics driver 

No apport report written because the error message indicates its a followup error from a previous failure.
No apport report written because the error message indicates its a followup error from a previous failure.


dpkg: dependency problems prevent configuration of cuda-demo-suite-12-2:
 cuda-demo-suite-12-2 depends on cuda-runtime-12-2; however:
  Package cuda-runtime-12-2 is not configured yet.

dpkg: error processing package cuda-demo-suite-12-2 (--configure):
 dependency problems - leaving unconfigured


No apport report written because MaxReports is reached already
No apport report written because MaxReports is reached already


Processing triggers for initramfs-tools (0.140ubuntu13.4) ...


No apport report written because MaxReports is reached already
No apport report written because MaxReports is reached already


update-initramfs: Generating /boot/initrd.img-5.15.0-124-generic
Errors were encountered while processing:
 nvidia-dkms-545
 cuda-drivers-545
 cuda-drivers
 nvidia-driver-545
 cuda-runtime-12-2
 cuda-12-2
 cuda-demo-suite-12-2
needrestart is being skipped since dpkg has failed


E: Sub-process /usr/bin/dpkg returned an error code (1)


UnexpectedExit: Encountered a bad command exit code!

Command: 'sudo apt -y install python3-pip python3-dev'

Exit code: 100

Stdout: already printed

Stderr: already printed



In [35]:
node.run('sudo rm /var/lib/dpkg/lock')
node.run('sudo rm /var/lib/dpkg/lock-frontend')
node.run('sudo rm /var/cache/apt/archives/lock')

<Result cmd='sudo rm /var/cache/apt/archives/lock' exited=0>

In [37]:
node.run('sudo rm /var/crash/nvidia-kernel-source-545.0.crash')

<Result cmd='sudo rm /var/crash/nvidia-kernel-source-545.0.crash' exited=0>

In [38]:
node.run('sudo apt-get remove --purge "^nvidia-.*" cuda-*')
node.run('sudo apt-get autoremove')
node.run('sudo apt-get autoclean')

Reading package lists...
Building dependency tree...
Reading state information...
Package 'nvidia-egl-wayland-common' is not installed, so not removed
Package 'nvidia-390' is not installed, so not removed
Package 'nvidia-libopencl1-dev' is not installed, so not removed
Package 'nvidia-current' is not installed, so not removed
Package 'nvidia-current-updates' is not installed, so not removed
Package 'nvidia-libopencl1' is not installed, so not removed
Package 'nvidia-driver-410' is not installed, so not removed
Package 'nvidia-387' is not installed, so not removed
Package 'nvidia-387-updates' is not installed, so not removed
Package 'nvidia-experimental-387' is not installed, so not removed
Package 'nvidia-384-updates' is not installed, so not removed
Package 'nvidia-experimental-384' is not installed, so not removed
Package 'nvidia-381' is not installed, so not removed
Package 'nvidia-381-updates' is not installed, so not removed
Package 'nvidia-experimental-381' is not installed, so n

E: Unable to locate package cuda-keyring_1.0-1_all.deb
E: Couldn't find any package by glob 'cuda-keyring_1.0-1_all.deb'
E: Couldn't find any package by regex 'cuda-keyring_1.0-1_all.deb'
E: Unable to locate package cuda-keyring_1.0-1_all.deb.1
E: Couldn't find any package by glob 'cuda-keyring_1.0-1_all.deb.1'
E: Couldn't find any package by regex 'cuda-keyring_1.0-1_all.deb.1'
E: Unable to locate package cuda-keyring_1.0-1_all.deb.2
E: Couldn't find any package by glob 'cuda-keyring_1.0-1_all.deb.2'
E: Couldn't find any package by regex 'cuda-keyring_1.0-1_all.deb.2'


moved
Package 'nvidia-experimental-313' is not installed, so not removed
Package 'nvidia-310' is not installed, so not removed
Package 'nvidia-310-updates' is not installed, so not removed
Package 'nvidia-experimental-310' is not installed, so not removed
Package 'nvidia-304' is not installed, so not removed
Package 'nvidia-304-updates' is not installed, so not removed
Package 'nvidia-experimental-304' is not installed, so not removed
Package 'nvidia-legacy-390xx-opencl-icd' is not installed, so not removed
Package 'nvidia-legacy-390xx-smi' is not installed, so not removed
Package 'nvidia-cuda-doc' is not installed, so not removed
Package 'nvidia-firmware-535-server-535.113.01' is not installed, so not removed
Package 'nvidia-firmware-535-server-535.146.02' is not installed, so not removed
Package 'nvidia-firmware-535-server-535.171.04' is not installed, so not removed
Package 'nvidia-firmware-535-535.104.12' is not installed, so not removed
Package 'nvidia-firmware-535-535.161.08' is 

UnexpectedExit: Encountered a bad command exit code!

Command: 'sudo apt-get remove --purge "^nvidia-.*" cuda-*'

Exit code: 100

Stdout: already printed

Stderr: already printed



In [32]:
node.run('wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb')
node.run('sudo dpkg -i cuda-keyring_1.0-1_all.deb')
node.run('sudo apt update')
node.run('sudo apt -y install linux-headers-$(uname -r)')
node.run('sudo apt -y install nvidia-driver-545')

--2024-10-29 23:06:53--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4328 (4.2K) [application/x-deb]
Saving to: ‘cuda-keyring_1.0-1_all.deb.2’

     0K ....                                                  100%  117M=0s

2024-10-29 23:06:53 (117 MB/s) - ‘cuda-keyring_1.0-1_all.deb.2’ saved [4328/4328]



(Reading database ... 107085 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.0-1_all.deb ...
Unpacking cuda-keyring (1.0-1) over (1.0-1) ...
Setting up cuda-keyring (1.0-1) ...


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 525 kB in 1s (402 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
8 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
linux-headers-5.15.0-124-generic is already the newest version (5.15.0-124.134).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
7 not fully installed or removed.
After this operation, 0 B of additional disk space will be used.
Setting up nvidia-dkms-545 (545.23.08-0ubuntu1) ...
update-initramfs: deferring update (trigger activated)

A modprobe blacklist file has been created at /etc/modprobe.d to prevent Nouveau
from loading. This can be reverted by deleting the following file:
/etc/modprobe.d/nvidia-graphics-drivers.conf

A new initrd image has also been created. To revert, please regenerate your
initrd by running the following command after deleting the modprobe.d file:
`/usr/sbin/initramfs -u`

*****************************************************************************
*** Reboot your computer and verify that the NVIDIA graphics driver can   ***
*** be loaded.                         

No apport report written because the error message indicates its a followup error from a previous failure.
No apport report written because the error message indicates its a followup error from a previous failure.
No apport report written because MaxReports is reached already
No apport report written because MaxReports is reached already
No apport report written because MaxReports is reached already
No apport report written because MaxReports is reached already


dpkg: dependency problems prevent configuration of nvidia-driver-545:
 nvidia-driver-545 depends on nvidia-dkms-545 (= 545.23.08-0ubuntu1); however:
  Package nvidia-dkms-545 is not configured yet.

dpkg: error processing package nvidia-driver-545 (--configure):
 dependency problems - leaving unconfigured
dpkg: dependency problems prevent configuration of cuda-runtime-12-2:
 cuda-runtime-12-2 depends on cuda-drivers (>= 535.104.05); however:
  Package cuda-drivers is not configured yet.

dpkg: error processing package cuda-runtime-12-2 (--configure):
 dependency problems - leaving unconfigured
dpkg: dependency problems prevent configuration of cuda-12-2:
 cuda-12-2 depends on cuda-runtime-12-2 (>= 12.2.2); however:
  Package cuda-runtime-12-2 is not configured yet.

dpkg: error processing package cuda-12-2 (--configure):
 dependency problems - leaving unconfigured
dpkg: dependency problems prevent configuration of cuda-demo-suite-12-2:
 cuda-demo-suite-12-2 depends on cuda-runtime-12-2

E: Sub-process /usr/bin/dpkg returned an error code (1)


UnexpectedExit: Encountered a bad command exit code!

Command: 'sudo apt -y install linux-headers-$(uname -r)'

Exit code: 100

Stdout: already printed

Stderr: already printed



In [21]:
try:
    node.run('sudo reboot') # reboot and wait for it to come up
except:
    pass
server.wait_for_tcp(reserved_fip, port=22)

In [22]:
node = ssh.Remote(reserved_fip) 
node.run('sudo apt -y install cuda-12-2 cuda-runtime-12-2 cuda-drivers=545.23.08-1')
node.run('sudo apt -y install nvidia-gds-12-2') # install instructions say to do this separately!
node.run('sudo apt -y install libcudnn8=8.9.6.50-1+cuda12.2 cuda-toolkit-12-2') # make sure the get cuda-11-8 version

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  adwaita-icon-theme ca-certificates-java cuda-cccl-12-2
  cuda-command-line-tools-12-2 cuda-compiler-12-2 cuda-crt-12-2
  cuda-cudart-12-2 cuda-cudart-dev-12-2 cuda-cuobjdump-12-2 cuda-cupti-12-2
  cuda-cupti-dev-12-2 cuda-cuxxfilt-12-2 cuda-demo-suite-12-2
  cuda-documentation-12-2 cuda-driver-dev-12-2 cuda-drivers-545 cuda-gdb-12-2
  cuda-libraries-12-2 cuda-libraries-dev-12-2 cuda-nsight-12-2
  cuda-nsight-compute-12-2 cuda-nsight-systems-12-2 cuda-nvcc-12-2
  cuda-nvdisasm-12-2 cuda-nvml-dev-12-2 cuda-nvprof-12-2 cuda-nvprune-12-2
  cuda-nvrtc-12-2 cuda-nvrtc-dev-12-2 cuda-nvtx-12-2 cuda-nvvm-12-2
  cuda-nvvp-12-2 cuda-opencl-12-2 cuda-opencl-dev-12-2 cuda-profiler-api-12-2
  cuda-sanitizer-12-2 cuda-toolkit-12-2 cuda-toolkit-12-2-config-common
  cuda-toolkit-12-config-common cuda-toolkit-config-common cuda-tools-12-2
  cuda-visual-tools-12-2 dconf

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 2882 MB in 52s (55.6 MB/s)
Selecting previously unselected package hicolor-icon-theme.
(Reading database ... 86021 files and directories currently installed.)
Preparing to unpack .../000-hicolor-icon-theme_0.17-2_all.deb ...
Unpacking hicolor-icon-theme (0.17-2) ...
Selecting previously unselected package libgdk-pixbuf2.0-common.
Preparing to unpack .../001-libgdk-pixbuf2.0-common_2.42.8+dfsg-1ubuntu0.3_all.deb ...
Unpacking libgdk-pixbuf2.0-common (2.42.8+dfsg-1ubuntu0.3) ...
Selecting previously unselected package libgdk-pixbuf-2.0-0:amd64.
Preparing to unpack .../002-libgdk-pixbuf-2.0-0_2.42.8+dfsg-1ubuntu0.3_amd64.deb ...
Unpacking libgdk-pixbuf-2.0-0:amd64 (2.42.8+dfsg-1ubuntu0.3) ...
Selecting previously unselected package gtk-update-icon-cache.
Preparing to unpack .../003-gtk-update-icon-cache_3.24.33-1ubuntu2.2_amd64.deb ...
Unpacking gtk-update-icon-cache (3.24.33-1ubuntu2.2) ...
Selecting previously unselected package humanity-icon-theme.
Preparing to unpack .../004-h

No apport report written because the error message indicates its a followup error from a previous failure.


Setting up cuda-cuobjdump-12-2 (12.2.140-1) ...
Setting up cuda-nvrtc-12-2 (12.2.140-1) ...
Setting up cuda-sanitizer-12-2 (12.2.140-1) ...
Setting up libavahi-common-data:amd64 (0.8-5ubuntu5.2) ...
Setting up cuda-nvvm-12-2 (12.2.140-1) ...
Setting up libxinerama1:amd64 (2:1.1.4-3) ...
Setting up cuda-cupti-12-2 (12.2.142-1) ...
Setting up libxrandr2:amd64 (2:1.5.2-1build1) ...
dpkg: dependency problems prevent configuration of cuda-drivers:
 cuda-drivers depends on cuda-drivers-545 (= 545.23.08-1); however:
  Package cuda-drivers-545 is not configured yet.

dpkg: error processing package cuda-drivers (--configure):
 dependency problems - leaving unconfigured
Setting up cuda-nvprof-12-2 (12.2.142-1) ...


...

Setting up libavahi-client3:amd64 (0.8-5ubuntu5.2) ...
dpkg: dependency problems prevent configuration of cuda-12-2:
 cuda-12-2 depends on cuda-runtime-12-2 (>= 12.2.2); however:
  Package cuda-runtime-12-2 is not configured yet.

dpkg: error processing package cuda-12-2 (--configure):
 dependency problems - leaving unconfigured
Setting up cuda-libraries-12-2 (12.2.2-1) ...
Setting up fontconfig (2.13.1-4.2ubuntu5) ...
Regenerating fonts cache... done.
Setting up libcurand-dev-12-2 (10.3.3.141-1) ...
Setting up libcusolver-dev-12-2 (11.5.2.141-1) ...
Setting up nsight-systems-2023.2.3 (2023.2.3.1004-33186433v0) ...
update-alternatives: using /opt/nvidia/nsight-systems/2023.2.3/target-linux-x64/nsys to provide /usr/local/bin/nsys (nsys) in auto mode
update-alternatives: using /opt/nvidia/nsight-systems/2023.2.3/host-linux-x64/nsys-ui to provide /usr/local/bin/nsys-ui (nsys-ui) in auto mode
Setting up dconf-gsettings-backend:amd64 (0.40.0-3) ...
dpkg: dependency problems prevent configur

No apport report written because MaxReports is reached already


Setting up libcublas-dev-12-2 (12.2.5.6-1) ...
Setting up cuda-crt-12-2 (12.2.140-1) ...
Setting up libcairo2:amd64 (1.16.0-5ubuntu2) ...
Setting up libcusparse-dev-12-2 (12.1.2.141-1) ...
Setting up cuda-nvcc-12-2 (12.2.140-1) ...
Setting up libnpp-dev-12-2 (12.2.1.4-1) ...
Setting up libcups2:amd64 (2.4.1op1-1ubuntu4.11) ...
Setting up libcairo-gobject2:amd64 (1.16.0-5ubuntu2) ...
Setting up libpangoft2-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Setting up cuda-compiler-12-2 (12.2.2-1) ...
Setting up libgtk-3-common (3.24.33-1ubuntu2.2) ...
Setting up libpangocairo-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Setting up cuda-nsight-systems-12-2 (12.2.2-1) ...
Setting up cuda-libraries-dev-12-2 (12.2.2-1) ...
Setting up adwaita-icon-theme (41.0-1ubuntu1) ...
update-alternatives: using /usr/share/icons/Adwaita/cursor.theme to provide /usr/share/icons/default/index.theme (x-cursor-theme) in auto mode
Setting up default-jre-headless (2:1.11-72build2) ...
Setting up libgtk2.0-0:amd64 (2.24.33-2ubuntu2.

E: Sub-process /usr/bin/dpkg returned an error code (1)


UnexpectedExit: Encountered a bad command exit code!

Command: 'sudo apt -y install cuda-12-2 cuda-runtime-12-2 cuda-drivers=545.23.08-1'

Exit code: 100

Stdout: already printed

Stderr: already printed



In [23]:
node.run("echo 'PATH=\"/usr/local/cuda-12.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin\"' | sudo tee /etc/environment")

PATH="/usr/local/cuda-12.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin"


<Result cmd='echo \'PATH="/usr/local/cuda-12.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin"\' | sudo tee /etc/environment' exited=0>

Now we have to reboot, and make sure we have the specified CUDA:

In [24]:
try:
    node.run('sudo reboot')
except:
    pass
server.wait_for_tcp(reserved_fip, port=22)

In [25]:
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new PATH
node.run('nvidia-smi')
node.run('nvcc --version')

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



UnexpectedExit: Encountered a bad command exit code!

Command: 'nvidia-smi'

Exit code: 9

Stdout: already printed

Stderr: already printed



Finally, add all SSH keys in your account at this site - 

In [26]:
nova=chi.clients.nova()
# iterate over all keypairs in this account
for kp in nova.keypairs.list(): 
    public_key = nova.keypairs.get(kp.name).public_key 
    node.run(f"echo {public_key} >> ~/.ssh/authorized_keys")

#### Optional: Install Python packages

Before starting your Jupyter instance, you may want to install some Python packages.

The following cell will install the *same* version of some key deep learning packages as are installed on Colab (as of November 2022), for maximum cross-compatibility. 

In [27]:
node.run('python3 -m pip install --user Cython==3.0.5')
node.run('wget https://raw.githubusercontent.com/teaching-on-testbeds/colab/main/requirements_chameleon_dl22.txt -O requirements_chameleon_dl22.txt')
node.run('python3 -m pip install --user -r requirements_chameleon_dl22.txt --extra-index-url https://download.pytorch.org/whl/cu118 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


--2024-10-29 22:56:08--  https://raw.githubusercontent.com/teaching-on-testbeds/colab/main/requirements_chameleon_dl22.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 719 [text/plain]
Saving to: ‘requirements_chameleon_dl22.txt’

     0K                                                       100% 26.3M=0s

2024-10-29 22:56:08 (26.3 MB/s) - ‘requirements_chameleon_dl22.txt’ saved [719/719]



Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 121.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 181.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

<Result cmd='python3 -m pip install --user -r requirements_chameleon_dl22.txt --extra-index-url https://download.pytorch.org/whl/cu118 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html' exited=0>

If you need additional packages, you can install them in a similar manner.

Test your installation - make sure Tensorflow, Pytorch, and JAX can all see the GPU:

In [82]:
node.run('python3 -c \'import tensorflow as tf; print(tf.config.list_physical_devices("GPU"))\'')
# should say: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

KeyboardInterrupt: 

In [30]:
node.run('python3 -c \'import torch; print(torch.cuda.get_device_name(0))\'')
# should say: Quadro RTX 6000

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/cc/.local/lib/python3.10/site-packages/torch/cuda/__init__.py", line 419, in get_device_name
    return get_device_properties(device).name
  File "/home/cc/.local/lib/python3.10/site-packages/torch/cuda/__init__.py", line 449, in get_device_properties
    _lazy_init()  # will define _get_device_properties
  File "/home/cc/.local/lib/python3.10/site-packages/torch/cuda/__init__.py", line 298, in _lazy_init
    torch._C._cuda_init()
RuntimeError: No CUDA GPUs are available


UnexpectedExit: Encountered a bad command exit code!

Command: "python3 -c 'import torch; print(torch.cuda.get_device_name(0))'"

Exit code: 1

Stdout: already printed

Stderr: already printed



In [ ]:
node.run('python3 -c \'import jax; print(jax.devices())\'')
# should say: cuda(id=0)

### Set up Jupyter on server

Install `jupyter_http_over_ws`, which is required in order to connect Colab to this Jupyter instance:

In [42]:
node.run('python3 -m pip install --user jupyter-core==5.5.0 jupyter-client==6.1.12 jupyter-server==1.24.0 jupyterlab-widgets==3.0.9  jupyterlab==3.6.6 jupyter_http_over_ws traitlets')

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.8/529.8 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.9/819.9 kB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 183.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 98.9 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

<Result cmd='python3 -m pip install --user jupyter-core==5.5.0 jupyter-client==6.1.12 jupyter-server==1.24.0 jupyterlab-widgets==3.0.9  jupyterlab==3.6.6 jupyter_http_over_ws traitlets' exited=0>

And, activate `jupyter_http_over_ws` - the output should show `jupyter_http_over_ws 0.0.7 OK` (don't worry if it also says "Validation failed"):

In [43]:
node.run('/home/cc/.local/bin/jupyter server extension enable --py jupyter_http_over_ws')

Enabling: jupyter_http_over_ws
- Writing config: /usr/etc/jupyter
    - Validating jupyter_http_over_ws...
      jupyter_http_over_ws 0.0.7 OK
      X Validation failed: [Errno 13] Permission denied: '/usr/etc'


<Result cmd='/home/cc/.local/bin/jupyter server extension enable --py jupyter_http_over_ws' exited=0>

In a **local terminal on your own laptop**, run

In [ ]:
print('ssh -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

to set up a tunnel to the Jupyter server.

 If your Chameleon key is not in the default location, you should also specify the path to your key as an argument, using `-i`. For example:

In [ ]:
print('ssh -i ~/.ssh/id_rsa_chameleon -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

Leave this SSH session open.

Then, run the following cell, which will run a command that does not terminate: 

In [ ]:
node.run("/home/cc/.local/bin/jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0")

In the output of the cell above, look for a URL in this format:
    
```
http://localhost:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

Copy this URL - you will need it in the next step.

Now, you can open Colab in a browser. Click on the drop-down menu for "Connect" in the top right and select "Connect to a local runtime". Paste the URL you copied earlier into the space and click "Connect". Your notebook should now be running on your Colab host (you can put `!hostname` in a cell and run it to verify!)


## Get Data

Download Food-11 Images

In [44]:
node.run('python3 -m pip install gdown')

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


<Result cmd='python3 -m pip install gdown' exited=0>

In [ ]:
ls -l content/Food-11

In [ ]:
node.run('ls -l content/Food-11')

In [71]:
node.run('~/.local/bin/gdown https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq -O content/Food-11/dataset.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq
From (redirected): https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq&confirm=t&uuid=16e0700a-2c0d-4226-bc09-2b02744be7ef
To: /home/cc/content/Food-11/dataset.zip
100%|██████████| 1.16G/1.16G [00:23<00:00, 49.2MB/s]


<Result cmd='~/.local/bin/gdown https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq -O content/Food-11/dataset.zip' exited=0>

In [62]:
node.run('~/.local/bin/gdown https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq -O content/Food-11')

Downloading...
From (original): https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq
From (redirected): https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq&confirm=t&uuid=5f19310a-0e2e-406c-8853-bb84216679c1
To: /home/cc/content/Food-11
100%|██████████| 1.16G/1.16G [00:14<00:00, 79.8MB/s]


<Result cmd='~/.local/bin/gdown https://drive.google.com/uc?id=1dt3CD3ICdLbTf80sNJ25TPBDKu_qyCnq -O content/Food-11' exited=0>

In [78]:
node.run('python3 -c "import zipfile; zipfile.ZipFile(\'content/Food-11/dataset.zip\', \'r\').extractall(\'content/Food-11\')"')

<Result cmd='python3 -c "import zipfile; zipfile.ZipFile(\'content/Food-11/dataset.zip\', \'r\').extractall(\'content/Food-11\')"' exited=0>

In [79]:
node.run('ls -l content/Food-11')

total 2271544
-rw-rw-r-- 1 cc cc 1162787257 Oct 29 23:31 Food-119jan1l1d.part
-rw-rw-r-- 1 cc cc 1162787257 Dec  4  2023 dataset.zip
drwxrwxr-x 2 cc cc      94208 Oct 30 00:06 evaluation
drwxrwxr-x 2 cc cc     266240 Oct 30 00:06 training
drwxrwxr-x 2 cc cc     102400 Oct 30 00:06 validation


<Result cmd='ls -l content/Food-11' exited=0>

In [80]:
node.run('ls -l content/Food-11/evaluation')
node.run('ls -l content/Food-11/training')
node.run('ls -l content/Food-11/validation')


total 250076
-rw-rw-r-- 1 cc cc   50352 Oct 30 00:06 0_0.jpg
-rw-rw-r-- 1 cc cc   39414 Oct 30 00:06 0_1.jpg
-rw-rw-r-- 1 cc cc   44843 Oct 30 00:06 0_10.jpg
-rw-rw-r-- 1 cc cc   55457 Oct 30 00:06 0_100.jpg
-rw-rw-r-- 1 cc cc   56862 Oct 30 00:06 0_101.jpg
-rw-rw-r-- 1 cc cc   61976 Oct 30 00:06 0_102.jpg
-rw-rw-r-- 1 cc cc   48582 Oct 30 00:06 0_103.jpg
-rw-rw-r-- 1 cc cc   63436 Oct 30 00:06 0_104.jpg
-rw-rw-r-- 1 cc cc   42097 Oct 30 00:06 0_105.jpg
-rw-rw-r-- 1 cc cc   62859 Oct 30 00:06 0_106.jpg
-rw-rw-r-- 1 cc cc   58849 Oct 30 00:06 0_107.jpg
-rw-rw-r-- 1 cc cc   49312 Oct 30 00:06 0_108.jpg
-rw-rw-r-- 1 cc cc   43813 Oct 30 00:06 0_109.jpg
-rw-rw-r-- 1 cc cc   49168 Oct 30 00:06 0_11.jpg
-rw-rw-r-- 1 cc cc   50190 Oct 30 00:06 0_110.jpg
-rw-rw-r-- 1 cc cc   24939 Oct 30 00:06 0_111.jpg
-rw-rw-r-- 1 cc cc   55618 Oct 30 00:06 0_112.jpg
-rw-rw-r-- 1 cc cc   46650 Oct 30 00:06 0_113.jpg
-rw-rw-r-- 1 cc cc   31216 Oct 30 00:06 0_114.jpg
-rw-rw-r-- 1 cc cc   47582 Oct 30 00:06 0_1

<Result cmd='ls -l content/Food-11/validation' exited=0>

## Get Repo

Clone Repository: https://github.com/tin2294/image-classification-continuous-x

In [ ]:
node.run('cd work && git clone git@github.com:tin2294/image-classification-continuous-x.git');

## Release resources

If you finish with your experimentation before your lease expires,release your resources and tear down your environment by running the following (commented out to prevent accidental deletions).

This section is designed to work as a "standalone" portion - you can come back to this notebook, ignore the top part, and just run this section to delete your reasources.

In [ ]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)
username = os.getenv('USER') # all exp resources will have this prefix

NODE_TYPE = "gpu_rtx_6000"
lease = chi.lease.get_lease(f"colab-{username}-{NODE_TYPE}")

In [ ]:
DELETE = False
# DELETE = True  # un-comment if you're ready to delete resources

if DELETE:

    # get info about resources
    server_id = chi.server.get_server_id(f"colab-{username}-{NODE_TYPE}")
    reserved_fip = [d['addr'] for d in chi.server.show_server(server_id).addresses['sharednet1'] if d['OS-EXT-IPS:type']=='floating'][0]

    # delete server   
    chi.server.delete_server(server_id)

    # release floating IP
    reserved_fip =  chi.lease.get_reserved_floating_ips(lease["id"])[0]
    ip_info = chi.network.get_floating_ip(reserved_fip)
    chi.neutron().delete_floatingip(ip_info["id"])

    # delete lease
    chi.lease.delete_lease(lease["id"])
